Nine healthy male subjects provided plasma samples. They underwent 4
regimens:

1. Regimen A: food+, exercise-
2. Regimen B: food-, exercise+
3. Regimen C: food+, exercise+
4. Regimen D: food-, exercise-


In [76]:
from isatools.create.models import *
from isatools.create.models import MSAssayTopologyModifiers

In [77]:
food = StudyFactor(name='food')
exercise = StudyFactor(name='exercise')

In [78]:
treatmentA = Treatment(treatment_type=INTERVENTIONS['CHEMICAL'],
    factor_values=(
        FactorValue(factor_name=food, value='food+'),
        FactorValue(factor_name=exercise, value='exercise-'),
    ))

treatmentB = Treatment(treatment_type=INTERVENTIONS['CHEMICAL'],
    factor_values=(
        FactorValue(factor_name=food, value='food-'),
        FactorValue(factor_name=exercise, value='exercise+'),
    ))

treatmentC = Treatment(treatment_type=INTERVENTIONS['CHEMICAL'],
    factor_values=(
        FactorValue(factor_name=food, value='food+'),
        FactorValue(factor_name=exercise, value='exercise+'),
    ))


treatmentD = Treatment(treatment_type=INTERVENTIONS['CHEMICAL'],
    factor_values=(
        FactorValue(factor_name=food, value='food-'),
        FactorValue(factor_name=exercise, value='exercise-'),
    ))


treatment_sequence_AB = TreatmentSequence(ranked_treatments= [(treatmentA,1), (treatmentB, 2)])
treatment_sequence_CD = TreatmentSequence(ranked_treatments= [(treatmentC,1), (treatmentD, 2)])

            

In [79]:
sample_assay_plan = SampleAssayPlan()
sample_assay_plan.group_size = 9

sample_assay_plan.add_sample_type('plasma')

sample_assay_plan.add_sample_plan_record('plasma', 1)

isa_factory = IsaModelObjectFactory(
            sample_assay_plan=sample_assay_plan,
            treatment_sequence=treatment_sequence_AB)
study = isa_factory.create_study_from_plan()

ms_assay_type = AssayType(measurement_type='metabolite profiling',
                                  technology_type='mass spectrometry')
ms_assay_type.topology_modifiers = MSAssayTopologyModifiers(
            injection_modes={'FIA', 'LC'},
            acquisition_modes={'positive', 'negative'},
            technical_replicates=2
        )

ngs_assay_type = AssayType(
            measurement_type='nucleotide sequencing', technology_type='NGS')
ngs_assay_type.topology_modifiers = DNASeqAssayTopologyModifiers(
            technical_replicates=1, distinct_libraries=1
        )
sample_assay_plan.add_assay_type(ms_assay_type)
sample_assay_plan.add_assay_type(ngs_assay_type)
sample_assay_plan.add_assay_plan_record('plasma', ms_assay_type)
sample_assay_plan.add_assay_plan_record('plasma', ngs_assay_type)

ms_assay_type1 = AssayType(measurement_type='metabolite profiling',
                                   technology_type='mass spectrometry')
ms_assay_type1.topology_modifiers = MSAssayTopologyModifiers(
            injection_modes={'LC'},
            acquisition_modes={'negative'},
            technical_replicates=1
        )
sample_assay_plan.add_assay_type(ms_assay_type1)
ms_assay_type2 = AssayType(measurement_type='metabolite profiling',
                                   technology_type='mass spectrometry')
ms_assay_type2.topology_modifiers = MSAssayTopologyModifiers(
            injection_modes={'FIA'},
            acquisition_modes={'positive', 'negative'},
            technical_replicates=2
        )
sample_assay_plan.add_assay_type(ms_assay_type2)

sample_assay_plan.add_assay_type(ngs_assay_type)
study = isa_factory.create_assays_from_plan()

from isatools.isatab import dump_tables_to_dataframes as dumpdf
from qgrid import show_grid

isa_investigation = Investigation(identifier='inv_dtp101')
isa_study = isa_factory.create_study_from_plan()
isa_study.filename = 's_study.txt'
isa_investigation.studies = [isa_study]
dataframes = dumpdf(isa_investigation)
sample_table = next(iter(dataframes.values()))
show_grid(sample_table)

show_grid(dataframes['s_study.txt'])



A protocol with name "metabolite extraction" has already been declared in the study
A protocol with name "metabolite extraction" has already been declared in the study
A protocol with name "metabolite extraction" has already been declared in the study


In [80]:
isa_investigation.studies = [isa_factory.create_assays_from_plan()]
for assay in isa_investigation.studies[-1].assays:
    print('Assay generated: {0}, {1} samples, {2} processes, {3} data files'
          .format(assay.filename, len(assay.samples), len(assay.process_sequence), len(assay.data_files)))
dataframes = dumpdf(isa_investigation)



A protocol with name "metabolite extraction" has already been declared in the study
A protocol with name "metabolite extraction" has already been declared in the study
A protocol with name "metabolite extraction" has already been declared in the study
Assay generated: a_plasma_ms_LC_negative_assay.txt, 36 samples, 108 processes, 72 data files
Assay generated: a_plasma_ms_LC_positive_assay.txt, 36 samples, 108 processes, 72 data files
Assay generated: a_plasma_ms_FIA_negative_assay.txt, 36 samples, 108 processes, 72 data files
Assay generated: a_plasma_ms_FIA_positive_assay.txt, 36 samples, 108 processes, 72 data files
